In [ ]:
import csv, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
working_dir = os.path.join('E:/','Lumos_Data','20220531 - 256-092 12ca5 vs ACE2')
prots = ['12ca5','ACE2']
rt_lim = 300

files_peaks = ['ModelSelection_All_MS2s_12ca5.csv','ModelSelection_All_MS2s_ACE2.csv']
files_pyb = ['12ca5 Full Output.csv', 'ACE2 Full Output.csv']

In [ ]:
mzs_peaks = []
rts_peaks = []
ALCs_peaks = []

mzs_pyb = []
rts_pyb = []
es_pyb = []
ps_pyb = []
areas_pyb = []

all_found_mzs = []
all_found_rts = []
all_found_es = []
all_found_ps = []
all_found_ALCs = []
all_found_areas = []

all_notfound_mzs = []
all_notfound_rts = []
all_notfound_es = []
all_notfound_ps = []
all_notfound_areas = []

In [ ]:
for file in files_pyb:
    mzs_prot = []
    rts_prot = []
    es_prot = []
    ps_prot = []
    areas_prot = []
    with open(os.path.join(working_dir,file),mode='r',encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file,delimiter=',')
        for row in csv_reader:
            mzs_prot.append(float(row['mz']))
            rts_prot.append(float(row['RT']))
            es_prot.append(float(row['Escore']))
            ps_prot.append(float(row['p value']))
            areas_prot.append(row['areas'])
    mzs_pyb.append(mzs_prot)
    rts_pyb.append(rts_prot)
    es_pyb.append(es_prot)
    ps_pyb.append(ps_prot)
    areas_pyb.append(areas_prot)

for file in files_peaks:
    mzs_prot = []
    rts_prot = []
    ALCs_prot = []
    with open(os.path.join(working_dir,file),mode='r',encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file,delimiter=',')
        for row in csv_reader:
            mzs_prot.append(float(row['mz']))
            rts_prot.append(float(row['RT'])*60)
            ALCs_prot.append(float(row['ALC']))
    mzs_peaks.append(mzs_prot)
    rts_peaks.append(rts_prot)
    ALCs_peaks.append(ALCs_prot)

In [ ]:
for i,mz_peaks in enumerate(mzs_peaks):
    rt_peaks = rts_peaks[i]
    mz_pyb = mzs_pyb[i]
    rt_pyb = rts_pyb[i]
    e_pyb = es_pyb[i]
    p_pyb = ps_pyb[i]
    area_pyb = areas_pyb[i]
    ALCs = ALCs_peaks[i]

    mzs_found = []
    rts_found = []
    es_found = []
    ps_found = []
    areas_found = []
    ALCs_found = []

    mzs_notfound = []
    rts_notfound = []
    es_notfound = []
    ps_notfound = []
    areas_notfound = []
    is_feature_in = 0
    for j,mz in enumerate(mz_pyb):
        if np.round(mz,2) in np.round(mz_peaks,2):
            idx = np.where(np.round(mz_peaks,2) == np.round(mz,2))[0][0]
            rt_id_peaks = rt_peaks[idx]
            if (rt_pyb[j] - rt_lim < rt_id_peaks) and (rt_pyb[j] + rt_lim > rt_id_peaks):
                mzs_found.append(mz)
                rts_found.append(rt_pyb[j])
                es_found.append(e_pyb[j])
                ps_found.append(p_pyb[j])
                areas_found.append(area_pyb[j])
                ALCs_found.append(ALCs[idx])
                is_feature_in -= -1
            else:
                mzs_notfound.append(mz)
                rts_notfound.append(rt_pyb[j])
                es_notfound.append(e_pyb[j])
                ps_notfound.append(p_pyb[j])
                areas_notfound.append(area_pyb[j])
        else:
            mzs_notfound.append(mz)
            rts_notfound.append(rt_pyb[j])
            es_notfound.append(e_pyb[j])
            ps_notfound.append(p_pyb[j])
            areas_notfound.append(area_pyb[j])
    all_found_mzs.append(mzs_found)
    all_found_rts.append(rts_found)
    all_found_es.append(es_found)
    all_found_ps.append(ps_found)
    all_found_ALCs.append(ALCs_found)
    all_found_areas.append(areas_found)

    all_notfound_mzs.append(mzs_notfound)
    all_notfound_rts.append(rts_notfound)
    all_notfound_es.append(es_notfound)
    all_notfound_ps.append(ps_notfound)
    all_notfound_areas.append(areas_notfound)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 10))
# fig = plt.figure()  # for 3D plots
for i in range(len(prots)):
    e_f = all_found_es[i]
    p_f = [-np.log10(p) for p in all_found_ps[i]]
    e_nf = all_notfound_es[i]
    p_nf = [-np.log10(p) for p in all_notfound_ps[i]]
    ALCs_f = all_found_ALCs[i]

    ALCs_sorted = np.sort(ALCs_f)[::-1]
    idxs_sorted = np.argsort(ALCs_f)[::-1]
    e_f_sorted = [e_f[j] for j in idxs_sorted]
    p_f_sorted = [p_f[j] for j in idxs_sorted]

    #### 3D plotting stuff
    # ax = fig.add_subplot(1,2,i+1,projection='3d')
    # ax.scatter(e_f_sorted,p_f_sorted,ALCs_sorted,c=ALCs_sorted,cmap='gnuplot',linewidth=0.5)
    # ax.plot_trisurf(e_f_sorted,p_f_sorted,ALCs_sorted,cmap='gnuplot',edgecolor='none')
    #### end 3D plotting stuff

    axs[i].scatter(e_nf, p_nf, c='gray', label='Found in pyBinder')
    cb = axs[i].scatter(e_f_sorted, p_f_sorted, c=ALCs_f,alpha=1,cmap='gnuplot',label='Found in PEAKS and pyBinder',vmin=70)
    axs[i].set_xscale('linear')
    axs[i].set_xlim([0, 1])
    axs[i].set_yscale('linear')
    axs[i].set_title(f'Volcano plot for {prots[i]}', fontsize=24)
    axs[i].set_xlabel('Enrichment Score', fontsize=20)
    axs[i].set_ylabel('-Log10(P value)', fontsize=20)
    axs[i].legend(loc='best', fontsize=14)
    axs[i].tick_params(axis='both', which='major', labelsize=16)
fig.colorbar(cb,ax=axs.ravel().tolist(),label='ALC Score (%)',orientation='vertical')
figname = 'PEAKS sequences mapped to volcano all MS2.png'
plt.savefig(os.path.join(working_dir,figname))
plt.show()

In [ ]:
for i,scores in enumerate(all_found_es):
    df = pd.DataFrame(columns=('Compound', 'm/z','p value','specificity'))
    RT = all_found_rts[i]
    mz = all_found_mzs[i]
    ps = all_found_ps[i]
    areas = all_found_areas[i]
    for j,score in enumerate(scores):
        A = 'EScore: ' + str(np.round(score,2)) + ' RTime: ' + str(np.round(RT[j],3))
        B = np.round(mz[j],4)
        C = float(ps[j])
        D = prots[i]
        df.loc[j] = [A,B,C,D]
    df.to_csv(os.path.join(working_dir,prots[i] + ' Found pyB.csv'),index=False)

for i,scores in enumerate(all_notfound_es):
    df = pd.DataFrame(columns=('Compound', 'm/z','p value','specificity'))
    RT = all_notfound_rts[i]
    mz = all_notfound_mzs[i]
    ps = all_notfound_ps[i]
    for j,score in enumerate(scores):
        A = 'EScore: ' + str(np.round(score,2)) + ' RTime: ' + str(np.round(RT[j],3))
        B = np.round(mz[j],4)
        C = float(ps[j])
        D = prots[i]
        df.loc[j] = [A,B,C,D]
    df.to_csv(os.path.join(working_dir,prots[i] + ' Not Found pyB.csv'),index=False)

In [ ]:
### get stuff not found in PEAKS

all_found_mzs_peaks = []
all_found_rts_peaks = []
all_found_es_peaks = []
all_found_ps_peaks = []

all_notfound_mzs_peaks = []
all_notfound_rts_peaks = []
all_notfound_es_peaks = []
all_notfound_ps_peaks = []

for i,mz_peaks in enumerate(mzs_peaks):
    rt_peaks = rts_peaks[i]
    mz_pyb = mzs_pyb[i]
    rt_pyb = rts_pyb[i]
    e_pyb = es_pyb[i]
    p_pyb = ps_pyb[i]

    mzs_found_peaks = []
    rts_found_peaks = []

    mzs_notfound_peaks = []
    rts_notfound_peaks = []
    is_feature_in = 0
    for j,mz in enumerate(mz_peaks):
        if np.round(mz,2) in np.round(mz_pyb,2):
            idx = np.where(np.round(mz_pyb,2) == np.round(mz,2))[0][0]
            rt_id_pyb = rt_pyb[idx]
            if (rt_peaks[j] - rt_lim < rt_id_pyb) and (rt_peaks[j] + rt_lim > rt_id_pyb):
                mzs_found_peaks.append(mz)
                rts_found_peaks.append(rt_peaks[j])
                is_feature_in -= -1
            else:
                mzs_notfound_peaks.append(mz)
                rts_notfound_peaks.append(rt_peaks[j])
        else:
            mzs_notfound_peaks.append(mz)
            rts_notfound_peaks.append(rt_peaks[j])
    all_found_mzs_peaks.append(mzs_found_peaks)
    all_found_rts_peaks.append(rts_found_peaks)

    all_notfound_mzs_peaks.append(mzs_notfound_peaks)
    all_notfound_rts_peaks.append(rts_notfound_peaks)

In [ ]:
for i,mzs in enumerate(all_found_mzs_peaks):
    df = pd.DataFrame(columns=('RT', 'm/z','specificity'))
    RT = all_found_rts_peaks[i]
    for j,mz in enumerate(mzs):
        A = 'RTime: ' + str(np.round(RT[j],3))
        B = np.round(mz,4)
        C = prots[i]
        df.loc[j] = [A,B,C]
    df.to_csv(os.path.join(working_dir,prots[i] + ' Found PEAKS All ms2.csv'),index=False)

for i,mzs in enumerate(all_notfound_mzs_peaks):
    df = pd.DataFrame(columns=('RT', 'm/z','specificity'))
    RT = all_notfound_rts_peaks[i]
    for j,mz in enumerate(mzs):
        A = 'RTime: ' + str(np.round(RT[j],3))
        B = np.round(mz,4)
        C = prots[i]
        df.loc[j] = [A,B,C]
    df.to_csv(os.path.join(working_dir,prots[i] + ' Not Found PEAKS All ms2.csv'),index=False)